## 1. Summary

I work in a chinese big IT company. My recent projects are about network data analysis. I spend about 2 weeks on the competition.

This notebook contains the all steps to produce the my final submissing, public and private LB scores are: 0.997700 and 0.999485.

Anyway, I don't use the description or other complicate features because of time. So I think there is improvement definitely in future.

I used common packages in python world, but xlearn is a exception. It is a factorization machine algorithm package which is useful in ctr prediction and recommendation system. The input data is libsvm format. The transformation process is time consuming. The link is here https://github.com/aksnzhy/xlearn

## 2. data exploration

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats, integrate
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import xlearn as xl

%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)

In [ ]:
sales_train=pd.read_csv('raw_data/sales_train.csv.gz')
sales_test=pd.read_csv('raw_data/test.csv.gz')
items=pd.read_csv('raw_data/items.csv')
item_categories=pd.read_csv('raw_data/item_categories.csv')

# rename the column for simplicity
sales_train.rename(columns={'date_block_num':'block'},inplace=True) 

### train data summary

In [ ]:
sales_train.describe()

nearly 3 million records without missing values

shop_id and item_id are categorical variables

item_price, item_cnt_day are numerical

#### groupby shop, item, block to build target dataset

In [ ]:
sales_month=sales_train.groupby(['shop_id','item_id','block'],as_index=False).agg({'item_cnt_day':'sum','item_price':'mean'}).rename(columns={'item_cnt_day':'item_cnt_month'})
sales_month.describe()

#### explore item price

In [ ]:
items_avg_price=sales_month.groupby('item_id')['item_price'].mean()
f, ax = plt.subplots(figsize=(15, 6));
sns.distplot(items_avg_price,kde=False);

the prices above 20000 are scarce. so clip the high values

In [ ]:
u,l=np.percentile(items_avg_price,[0,95])
f, ax = plt.subplots(figsize=(15, 6))
sns.distplot(np.clip(items_avg_price,u,l),kde=False);

most prices are range from 0 to 500

In [ ]:
sns.jointplot(x='item_price',y='item_cnt_month',data=sales_month[sales_month.item_price<2000]);

item_price and item_month_cnt are not strong corralated

#### explore month block / time sequence

In [ ]:
f, ax = plt.subplots(figsize=(15, 6))
sns.barplot(x='block',y='item_cnt_month',data=sales_month.groupby('block',as_index=False)['item_cnt_month'].sum());

it seems the total sales decreese every month.

christmas months are spikes

In [ ]:
f, ax = plt.subplots(figsize=(15, 6))
sns.barplot(x='block',y='item_cnt_month',data=sales_month.groupby(['block'],as_index=False)['item_cnt_month'].count());

the month shop&item sale pair is about 40k average.

#### explore shops

In [ ]:
shop_sales=sales_month.groupby('shop_id',as_index=False)[['item_cnt_month']].sum().sort_values('item_cnt_month',ascending=False)
f, ax = plt.subplots(figsize=(15, 6))
sns.barplot(x='shop_id',y='item_cnt_month',data=shop_sales,order=shop_sales.shop_id);

shop 31 is the largest shop

near 2/3 shop sales cnt are close

#### explore items

In [ ]:
item_sales=sales_month.groupby('item_id',as_index=False)[['item_cnt_month']].sum().sort_values('item_cnt_month',ascending=False)
f, ax = plt.subplots(figsize=(15, 6))
sns.barplot(x='item_id',y='item_cnt_month',data=item_sales[:20],order=item_sales[:20].item_id);

In [ ]:
item_sales.describe()

it shows the avg item sale cnt is 167. below 75% is 124. some item like 20949 is very large which exceeds 170000

### explore test data

In [ ]:
sales_test.describe()

the test shop&item pairs is 210k, which is bigger too much corresponding 40k. 

so not every shop item pair will have sale cnt, e.g. some pairs' cnt should be zero.

In [ ]:
sns.barplot(data=sales_test.groupby(['shop_id'],as_index=False)[['item_id']].count(),x='shop_id',y='item_id');

the plot shows every shop has about 5000 items.

In [ ]:
sales_test.item_id.nunique()

gotcha! the total sale items is 5100. so the test set is produced by cross-products of shops and items in test month.

## 3.Feature engineering and Validation

now let's build train and validation set

train and validation data should be cross-producted like test set.

I hold the last month in train data as validation data.

As features, I use previous month sales, item category, shop month sales, item month sales.

In [ ]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
        use to compact dataset
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

def clip_y(y):
    '''
    predicted cnt value should be in [0,20]
    '''
    return np.clip(y,0,20)

def score(y_t,y_p):
    return mean_squared_error(clip_y(y_t),clip_y(y_p))**0.5

In [ ]:
%%time
label_col='item_cnt_month'

# the last train block is validate block
val_block=np.max(sales_train.block)

# cross product shops and items
sales_cross=[]
index_cols=['shop_id','item_id']
for d in sorted(sales_train.block.unique()):
    m_sales=sales_train[sales_train.block==d]
    m_si=pd.MultiIndex.from_product([m_sales['shop_id'].unique(),m_sales['item_id'].unique()],
                                          names=index_cols).to_frame(index=False)
    
    m_si=m_si.merge(m_sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'}),
                    on=index_cols,how='left').rename(columns={'item_cnt_day':label_col})
    m_si['block']=d
    sales_cross.append(m_si)


sales_test['block']=val_block+1

sales_cross.append(sales_test.drop('ID',axis=1))

# don't use 2013 data
sales=pd.concat(sales_cross)[lambda df:df.block>12]
sales.fillna(0,inplace=True)

sales_p=sales.copy()

# joined with shop month sales
shop_month_sales=sales_train.groupby(['shop_id','block'],as_index=False).agg({
    'item_id':'nunique',
    'item_cnt_day':'sum'
}).rename(columns={
    'item_id':'shop_item_id_cnt_month',
    'item_cnt_day':'shop_item_cnt_month'
})
shop_month_sales['shop_item_avg_cnt_month']=shop_month_sales['shop_item_cnt_month']/shop_month_sales['shop_item_id_cnt_month']
shop_month_sales.drop(['shop_item_cnt_month','shop_item_id_cnt_month'],axis=1,inplace=True)
sales_p=sales_p.merge(shop_month_sales,on=['shop_id','block'],how='left')
shop_month_cols=shop_month_sales.columns.difference(['shop_id','block'])

# joined with item month sales
item_month_sales=sales_train.groupby(['item_id','block'],as_index=False).agg({
    'shop_id':'nunique',
    'item_cnt_day':'sum',
    'item_price':lambda s:np.mean(s)
}).rename(columns={
    'shop_id':'item_shop_cnt_month',
    'item_cnt_day':'item_shop_sale_month'
})
item_month_sales['item_shop_avg_cnt_month']=item_month_sales['item_shop_sale_month']/item_month_sales['item_shop_cnt_month']
item_month_sales.drop(['item_shop_cnt_month','item_shop_sale_month'],axis=1,inplace=True)
sales_p=sales_p.merge(item_month_sales,on=['item_id','block'],how='left')
item_month_cols=item_month_sales.columns.difference(['item_id','block'])

# build previous month sales
index_cols=['shop_id','item_id','block']
merge_cols=sales_p.columns.difference(index_cols)
for i in [1,2,3,]:
    prev=sales_p[merge_cols.union(index_cols)].copy()
    prev.block+=i
    prev.rename(columns=lambda c:'prev_%s_%d'%(c,i) if c in merge_cols else c,inplace=True)
    sales_p=sales_p.merge(prev,on=index_cols,how='left')

sales_p=sales_p.merge(items.drop(['item_name'],axis=1),how='left',on='item_id')

# joined with item category
item_cats=items.groupby(['item_category_id'],as_index=False).agg({
    'item_id':'count'
}).rename(columns={'item_id':'item_cat_items_cnt'})
sales_p=sales_p.merge(item_cats,on='item_category_id',how='left')

sales_p=downcast_dtypes(sales_p)
sales_p.fillna(0,inplace=True)

# drop current month data
drop_cols=set([label_col])
ext_cols=[shop_month_cols,item_month_cols]
for ec in ext_cols:
    drop_cols = drop_cols|set(ec)

# split train, validation, test data
val_block=np.max(sales_p.block)-1
X_train,X_val,X_test,y_train,y_val,y_test=sales_p.drop(drop_cols,axis=1)[lambda df:df.block<val_block],\
sales_p.drop(drop_cols,axis=1)[lambda df:df.block==val_block],\
sales_p.drop(drop_cols,axis=1)[lambda df:df.block==val_block+1],\
sales_p[sales_p.block<val_block][label_col],\
sales_p[sales_p.block==val_block][label_col],\
sales_p[sales_p.block==val_block+1][label_col],\

cat_cols=['shop_id','item_id','item_category_id']

# after feature engineering and data split, use base predict values to test validation data
print('base predict score %.4f\n'%mean_squared_error(clip_y(y_val),np.ones(y_val.shape[0])*0.5)**0.5)
X_train.info()

## 4.train models

I use two models. lightgbm is tree model. xlearn is ffm model.

### 4.1 train by gbdt

lightgbm has early_stopping mechanism by providing validation data

I get validation score 0.9632

In [ ]:
%%time

lgb1=LGBMRegressor(n_jobs=8)
lgb1.fit(X_train,y_train,
        eval_set=(X_val,y_val),
        early_stopping_rounds=1,
        eval_metric=lambda y_t,y_p:('error',score(y_t,y_p),False),
       )

### 4.2 train by gbdt with mean encoding

lightgbm has target encoding mechanism for categorical variables

I remove the shop and item summary features, get validation score 0.9892

In [ ]:
%%time

drop_cols=[c for c in X_train.columns if c.startswith('prev_') and not c.startswith('prev_item_cnt_month')]
X_train_lgb2=X_train.copy()
X_val_lgb2=X_val.copy()
X_test_lgb2=X_test.copy()
for x in X_train_lgb2,X_val_lgb2,X_test_lgb2:
    x.drop(drop_cols,axis=1,inplace=True)

lgb2=LGBMRegressor(n_jobs=8)
lgb2.fit(X_train_lgb2,y_train,
        eval_set=(X_val_lgb2,y_val),
        early_stopping_rounds=1,
        eval_metric=lambda y_t,y_p:('error',score(y_t,y_p),False),
        categorical_feature=cat_cols,
       )

# y_pred_val_lgb2=clip_y(lgb2.predict(X_test))

### 4.3 train by ffm

ffm is useful in most categorical datasets. but transform dataset into libsvm format is very time consuming.

I get validation score 1.0032

In [ ]:
%%time
def df_to_ffm(x,y,file,cat_cols):
    x.index=np.arange(x.shape[0])
    y.index=np.arange(x.shape[0])
    cat_idx=set([-1])
    for i in range(x.shape[1]):
        if x.columns[i] in cat_cols:
            cat_idx.add(i)
            
    with open(file,'w') as f:
        for i in range(x.shape[0]):
            s=str(y[i])
            
            for j in range(x.shape[1]):
                if j in cat_idx:
                    s += ' %d:%d:1' % (j,x.iat[i,j])
                else:
                    if x.iat[i,j]<0:
                        s += ' %d:%d:1' % (j,0)
                    else:
                        s += ' %d:%d:1' % (j,x.iat[i,j])
                    
            f.write(s+'\n')

df_to_ffm(X_train,y_train,'ffm_data/train.ffm',[])
df_to_ffm(X_val,y_val,'ffm_data/val.ffm',[])
df_to_ffm(X_test,y_test,'ffm_data/test.ffm',[])

In [ ]:
%%time
ffm_model = xl.create_ffm()
ffm_model.setTrain("ffm_data/train.ffm")  # Training data
ffm_model.setTest('ffm_data/test.ffm')
ffm_model.setValidate("ffm_data/val.ffm")  # Validation data

param = {'task':'reg','epoch':15,'lr':1e-2,'k':4,'lambda':2e-3}

ffm_model.fit(param, 'ffm_data/model.out')

## 5 ensemble

I use the three models' validation results to train a ensemble regressor. 

I choose a gbdt to stacking the results to get LB 0.997700 and 0.999485.

In [ ]:
y_pred_val_lgb1=clip_y(lgb1.predict(X_val))
y_pred_test_lgb1=clip_y(lgb1.predict(X_test))

y_pred_val_lgb2=clip_y(lgb2.predict(X_val_lgb2))
y_pred_test_lgb2=clip_y(lgb2.predict(X_test_lgb2))

ffm_model.setTest('ffm_data/val.ffm')
ffm_model.predict('ffm_data/model.out','ensemble/ffm_val.csv')
y_pred_val_ffm=clip_y(pd.read_csv('ensemble/ffm_val.csv',header=None))

ffm_model.setTest('ffm_data/test.ffm')
ffm_model.predict('ffm_data/model.out','ensemble/ffm_tst.csv')
y_pred_test_ffm=clip_y(pd.read_csv('ensemble/ffm_tst.csv',header=None))

stacking_reg=LGBMRegressor()
stacking_reg.fit(np.c_[y_pred_val_lgb1,y_pred_val_lgb2,y_pred_val_ffm],y_val)
sales_test[label_col]=clip_y(stacking_reg.predict(np.c_[y_pred_test_lgb1,y_pred_test_lgb2,y_pred_test_ffm]))

sales_test[['ID',label_col]].to_csv('output/final.csv',index=False)